In [ ]:
#-------------------------------------------------------------------------------
#
# Run this with: 
#     voila --VoilaConfiguration.file_whitelist="['DUSTMONITOR.*', 'ACCESS.png', 'CITIES.png']"  Fidas_dashboard.ipynb
#
# Without whitelisting the logos in the 'about' tab won't appear, and the data file download  
# will not work (error 403 'forbidden').
# Note that within Jupyter notebook/lab only text file download will work (will be opened and
# visualized in a new tab). But Jupyter doesn't know how to handle .nc files, and so gives you
# a pop-up error with a silly message ('File download error: the file is not utf-8 encoded').
# Download will work in voilà, provided that the files have been correctly whitelisted.
# In the next cell the calls to 'display' may be commented out when working in Jupyter: they are
# meant to avoid excessive whitespace on the page margins when running the dashboard in voilà.
#

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.jp-Cell {padding: 0 !important; }</style>"))
display(HTML("<style>.jp-Notebook {padding: 0 !important; }</style>"))
from netCDF4 import Dataset
import datetime
import dateutil
from fnmatch import fnmatch
import ipywidgets as widgets
import os
import numpy as np
from IPython.display import display, FileLink
import pymongo
import pandas as pd
import xarray as xr

In [ ]:
# constant definitions
STATION = 'station1'
MONGO_IP = '10.224.83.51'
MONGO_PORT = 27017
DATABASE = 'stations'

In [ ]:
# connect to MongoDB
client = pymongo.MongoClient(host=MONGO_IP, port=MONGO_PORT, username='reader', password='12345')
db = client[DATABASE]

In [ ]:
class DataStore:

    def __init__(self, db: pymongo.database.Database, station_num: int = 1):

        self.station = db[f'station{station_num}']

        self.conf = db[f'station{station_num}'].find_one({'config': True})['sensors']
        self.all_measurements = self.get_all_measurements()
        self.data = self.query(self.all_measurements)
        self.datetime = self.data['datetime']

        self.measurements = self.get_measurement_names()
        
        self.create_config()

    def get_all_measurements(self):
        """
        Queries the DB for all measurements and returns a list of strings in form sensor+measurement+index, i.e. particulate_matter.PM1count.1
        """
        measurements = []
        doc = self.station.find({'month' : {"$exists" : True}})[0]

        for sensor_type in self.conf:
            if sensor_type not in ['gps']:

                for sens_ind in range(self.conf[sensor_type]):

                    for field in doc[sensor_type]:
                        if field != "type":

                            measurement = f"{sensor_type}.{field}.{sens_ind}"
                            measurements.append(measurement)

        return measurements 

    def query(self, measurements):
        month = {'$match': {'month': {'$exists': True}}}

        sort = {'$sort' : {'month': -1}}

        group = {'$group': {
                '_id': None, 
                'datetime': {'$push': '$gps.datetime'}
            }}

        names = []

        for measurement in measurements:
            vals = measurement.split(".")
            name = f"{vals[1]}+{vals[2]}" # the name is stored as "sensor+measurement+index", i.e. "particulate_matter+PM1mass+0"
            group['$group'][name] = {'$push' : f"${measurement}"}
            names.append(name)

        concat_arrays = {'$project': {
                '_id': 0, 
                'datetime': {'$reduce': {
                        'input': '$datetime', 
                        'initialValue': [], 
                        'in': {
                            '$concatArrays': ['$$this', '$$value']
                        }}}
                }}

        for name in names:
            concat_arrays['$project'][name] = {'$reduce' : { 
                'input' : f"${name}",
                'initialValue' : [],
                'in' : {
                    '$concatArrays': ['$$this', '$$value']
                }}} 

        aggr = self.station.aggregate([month, sort, group, concat_arrays])
        for x in iter(aggr):
            data_dict = x
        df = pd.DataFrame()
        for key in data_dict.keys():
            df[key] = data_dict[key]
            
        return df
    
    def get_measurement_names(self):
        measurements = set()
        for key in self.data:
            if key != 'datetime':
                measurements.add(f"{key.split('+')[0]}")

        return measurements

    def get_series(self, key):
        return self.data[key]
    
    def create_config(self):
        self.config = {}
        cols = self.data.columns
        
        for col in cols:
            if col in self.measurements:
                continue
            
            measure = col.split('+')[0]
            self.config[measure] = self.config.get(measure, 0) + 1

    def to_csv(self, start_date = None, end_date = None, cols = None):   
        
        if start_date == None:
            start_date = self['datetime'].iloc[0]
        if end_date == None:
            end_date = self['datetime'].iloc[-1]
            
        if cols == None:
            return self.all_data.set_index('datetime').loc[start_date:end_date,:].to_csv(index=True, header=True)
        
        return self.all_data.set_index('datetime').loc[start_date:end_date,cols].to_csv(index=True, header=True)
    
    def to_netcdf(self, start_date = None, end_date=None, cols=None):
        
        if start_date == None:
            start_date = self['datetime'].iloc[0]
        if end_date == None:
            end_date = self['datetime'].iloc[-1]
            
        if cols == None:
            x = xr.Dataset.from_dataframe(self.all_data.set_index('datetime').loc[start_date:end_date,:])
        else:
            x = xr.Dataset.from_dataframe(self.all_data.set_index('datetime').loc[start_date:end_date,cols])
            
        ### Per datum in the column, attributes need to be assigned, tentative list includes: full name, unit, sensor of origin, and various sensor specs
            
        return x.to_netcdf()
    
    def to_json(self, start_date = None, end_date = None, cols = None):
        
        if start_date == None:
            start_date = self['datetime'].iloc[0]
        if end_date == None:
            end_date = self['datetime'].iloc[-1]
            
        if cols == None:
            return self.all_data.set_index('datetime').loc[start_date:end_date,:].to_json(date_format = "iso")

        return self.all_data.set_index('datetime').loc[start_date:end_date,cols].to_json(date_format = "iso")
        
    def __getitem__(self, key):
        return self.all_data[key]

In [ ]:
class Plotter:
    '''
    Class to manage everything to do with plotting and plt
    '''
    
    DEFAULT_COLORS = {0: ['#8B0000', '#FF3131'],
                      1: ['#00008B', '#1F51FF'],
                      2: ['#008B00', '#39FF14'],
                      3: ['#8B8000', '#FFFF33']}
    
    def __init__(self, x_axis: iter, date_range_slider: widgets.SelectionRangeSlider) -> None:
        '''
        Initializes class
        @param x-axis Iterable containing x-axis elements. All plots managed by this plotter class must share
            the same x-axis
        @param date_range_slider Slider widget to control / limit the range of the x-axis
        '''
        
        # set up figure and plt settings
        plt.ioff()
        self.fig = plt.figure()
        self.fig.canvas.header_visible = False
        self.fig.canvas.resizable = False
        self.fig.canvas.toolbar_position = 'right'
        self.fig.canvas.layout.width = '100%'
        self.fig.set_figwidth(7)
        
        self.date_range_slider = date_range_slider
        
        # initialize variables
        self.x_axis = np.array(x_axis)
        self.axes = []
        self.colors = list(self.DEFAULT_COLORS.keys())  # keeps track of int for each default color
        self.max_graphs = len(self.colors)
        self.curr_graphs = 0


    def add_plot(self, data: iter, description: str) -> plt.Axes:
        '''
        Create a new subplot for the graph
        @param data Iterable of y-data to plot, len(data) must match len(self.x_axis)
        @param description What is being plotted, label for y-axis
        '''
        
        # check if we can add new plot
        # checks if data is compatible
        if (self.curr_graphs >= self.max_graphs) or \
            (data is None) or \
            (len(self.x_axis) != len(data)):
            return None
        
        if self.curr_graphs == 0:
            ax = self.fig.add_subplot()
        else:
            ax = self.axes[0].twinx()
            # pushes axis further away to not overlap
            ax.spines['right'].set_position(('outward', 
                                             50*(self.curr_graphs - 1)))
        
        # add graph description
        ax.description = description
        
        # plot
        ax.color = self.colors.pop()
        g_color = self.DEFAULT_COLORS[ax.color][0]  # new plots use the first color, subplots the second
        ax.plot(self.x_axis, data, '.',
               markersize=1, color=g_color)
        
        # edit axis info
        ax.set_ylabel(description, 
                        fontsize=12, color=g_color)
        ax.tick_params(axis='y', colors=g_color)
        self.fig.autofmt_xdate(rotation=45)
        
        
        self.curr_graphs += 1
        self.axes.append(ax)
        self.date_range_callback({'name': 'value'})
        return ax


    def add_subplot(self, data: iter, ax: plt.Axes) -> None:
        '''
        Adds new plot to an existing axis
        Only supports adding a single subplot per axes
        @param data Iterable of data to plot
        @param ax Existing plt.Axes object to graph
        '''
        g_color = self.DEFAULT_COLORS[ax.color][1]
        ax.plot(self.x_axis, data, '.', markersize=1, color=g_color)
        self.date_range_callback({'name': 'value'})


    def clear_plots(self) -> None:
        '''
        Clears all the plots and axes
        Resets the figure and list of available colors
        '''
        self.fig.clf()
        self.axes = []
        self.colors = list(self.DEFAULT_COLORS.keys())
        self.curr_graphs = 0
        self.date_range_callback({'name': 'value'})
    
    
    def date_range_callback(self, wdic: dict) -> None:
        '''
        Callback for date range slider to edid min/max dates on graph
        '''
        
        if wdic['name'] != 'value':
            return
        #The right end of the date range needs to be rounded up to the next day
        min_day = self.date_range_slider.value[0]
        max_day = self.date_range_slider.value[1] + datetime.timedelta(days=1)
        self.axes[0].set_xlim((min_day, max_day))
        self.finish_callback()
    
    
    def finish_callback(self):
        self.fig.tight_layout(pad=1.02)
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
    
        

In [ ]:
class ButtonList:
    '''
    This class groups all the buttons for different measurements of a sensor
    Manages the button callback functions and sending the appropriate info to the plotter object
    '''
    
    def __init__(self, data: DataStore, plotter: Plotter) -> None:
        '''
        Creates a new instance of a ButtonList
        @param data Instance of DataStore class. Contains the data to plot and also the attributes to 
            generate the buttons
        @param plotter Instance of Plotter class, manages the plotting of items
        '''
        
        # initiate class variables
        self.store = data
        self.plotter = plotter
        
        # collect info from DataStore object
        self.timeseries = self.store.datetime
        self.measurements = list(self.store.measurements)
        
        # init all buttons
        self.button_list = []
        self.active_buttons = []  # keeps track of currently active buttons
        for measurement in sorted(self.measurements):
            

            self.button_list.append(
                widgets.ToggleButton(
                    value = False,
                    description = measurement,
                    # tooltip=f"{self.store.data[ts][self.store.iLONG_NAME]} ({self.store.data[ts][self.store.iUNITS]})",
                    disabled=False,
                )
            )
            
            # add the callback function to the button
            self.button_list[-1].observe(self.callback)
            
            # button_list[-1]._Fidas_dashboard_units = self.store.data[ts][self.store.iUNITS]

        self.buttons = widgets.VBox(self.button_list)

        
    def callback(self, wdic: dict) -> None:
        '''
        Gets called when a button gets clicked
        Plots / clears the clicked button's measurments
        @param wdic Dictionary passed by the button. Contains at least the following keys:
            'type': type of notification
            If wdic['type'] is 'change' then the following keys are also passed
            'owner': the HasTraits instance
            'old': old value of the modified trait
            'new': new value of modified trait attribute
            'name': name of modified trait attribute
        '''
        
        # check if the trait changed is 'value'
        if wdic['name'] != 'value':
            return
        
        # check if data is being de-selected
        elif wdic['new'] == False:
            # remove element from the list of active buttons
            if wdic['owner'] in self.active_buttons:
                self.active_buttons.remove(wdic['owner'])
            else:
                return
            # clear plotter
            self.plotter.clear_plots()
            self.plot_graphs()  # plots all active buttons
            return
        
        # try and plot graph
        if not self.plot_graph(wdic['owner']):
            wdic['owner'].value = False  # change the value back to false if unable to plot it
        else:
            self.active_buttons.append(wdic['owner'])


    def plot_graphs(self) -> None:
        '''
        Plots all graphs in self.active_buttons
        '''
        for button in self.active_buttons:
            self.plot_graph(button)


    def plot_graph(self, button: widgets.ToggleButton) -> bool:
        '''
        Plots the data of the specified @param button
        If unable to plot it, returns False
        '''
        
        # get the attribute being plotted
        description = button.description
        
        # first collect the number of plots (1 or 2)
        num_plots = self.store.config.get(description, 0)
        
        # try plotting the first graph
        if num_plots == 0:
            # key isn't found in cofig
            ax = self.plotter.add_plot(self.store.data[description], description)
        else:
            # key is found in config
            ax = self.plotter.add_plot(self.store.data[f'{description}+0'], description)
            
            
        if ax is None:
            return False
        
        # try plotting subplot if needed
        if num_plots == 2:
            self.plotter.add_subplot(self.store.data[f'{description}+1'], ax)
        
        return True
        

In [ ]:
data = DataStore(db, 1)

slider_days = np.unique([x.date() for x in data.datetime])
date_range_slider = widgets.SelectionRangeSlider(
    options = slider_days,
    description = 'Date range:',
    orientation = 'horizontal',
    index = (0, len(slider_days)-1),
    disabled = False,
    continuous_update = False,
    tooltip = 'Select the date range to be plotted',
    layout=widgets.Layout(width='100%')
)

plotter = Plotter(data.datetime, date_range_slider)

date_range_slider.observe(plotter.date_range_callback)

button_list = ButtonList(data, plotter)

In [ ]:
decorated_canvas = widgets.VBox([date_range_slider,
                                 plotter.fig.canvas])
tab_time_series = widgets.HBox([button_list.buttons, decorated_canvas])

In [ ]:
#----------------------------------------------------------------------------------------------
#***Widgets for the intro/about tab***

In [ ]:
intro = widgets.HTML(
    value="""<p style="line-height: 150%">A Palas Fidas 200S aerosol spectrometer is operated at NYUAD by the 
    Arabian Center for Climate and Environmental Sciences, jointly with the Center 
    for Interacting Urban Networks. From the tabs above you can visualize current 
    and past measurements of dust concentration, as well as basic meteorological 
    parameters. You can also download the monthly data in netCDF4 or tabbed text format.</p>
    <p>&nbsp;</p>""",
    layout=widgets.Layout(width='700px')
)
logo_ACCESS = widgets.HTML(
    value='<img src="ACCESS.png" alt="Arabian Center for Climate and Environmental Sciences" style="width:300px">',
    layout=widgets.Layout(
        margin='0 20px 0 20px'
    )
)
logo_CITIES = widgets.HTML(
    value='<img src="CITIES.png" alt="Center for Interacting Urban Networks" style="width:300px">',
    layout=widgets.Layout(
        margin='0 20px 0 20px'
    )
)
tab_about = widgets.VBox([intro, widgets.HBox([logo_ACCESS, logo_CITIES])])

In [ ]:
#----------------------------------------------------------------------------------------------
#***Display the tabbed interface***

In [ ]:
tabbed_interface = widgets.Tab()
tabbed_interface.children = [tab_about, tab_time_series] #, tab_spectra, tab_downloads]
tabbed_interface.set_title(0, 'About')
tabbed_interface.set_title(1, 'Time series')
# tabbed_interface.set_title(2, 'Particle spectra')
# tabbed_interface.set_title(3, 'Data download')
display(tabbed_interface)